<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Steuerman-re-analysis" data-toc-modified-id="Steuerman-re-analysis-1">Steuerman re-analysis</a></span><ul class="toc-item"><li><span><a href="#Set-up-for-analysis" data-toc-modified-id="Set-up-for-analysis-1.1">Set up for analysis</a></span><ul class="toc-item"><li><span><a href="#Load-packages" data-toc-modified-id="Load-packages-1.1.1">Load packages</a></span></li><li><span><a href="#Specify-directories" data-toc-modified-id="Specify-directories-1.1.2">Specify directories</a></span></li></ul></li><li><span><a href="#Download-data" data-toc-modified-id="Download-data-1.2">Download data</a></span></li><li><span><a href="#Read-cell-gene-matrices" data-toc-modified-id="Read-cell-gene-matrices-1.3">Read cell-gene matrices</a></span></li></ul></li></ul></div>

# Steuerman re-analysis
Re-analyzes data from single-cell RNA-seq of cells from influenza-infected mice performed by [Steuerman et al (2018)](https://doi.org/10.1016/j.cels.2018.05.008).
The goal is to determine what fraction of cells are IFN+.

## Set up for analysis

### Load packages

In [1]:
options(warn=-1) # suppress warnings that clutter output

# install R packages
r_packages <- c(
  "ggplot2", "dplyr", "magrittr", "tidyverse")
suppressMessages(invisible(
  lapply(r_packages, library, character.only=TRUE)))

# install Bioconductor packages
bioc_packages <- c("GEOquery")
suppressMessages(invisible(
  lapply(bioc_packages, library, character.only=TRUE)))

sessionInfo()

R version 3.4.3 (2017-11-30)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Sierra 10.12.6

Matrix products: default
BLAS: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libBLAS.dylib
LAPACK: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libLAPACK.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] GEOquery_2.46.15    Biobase_2.38.0      BiocGenerics_0.24.0
 [4] forcats_0.2.0       stringr_1.2.0       purrr_0.2.4        
 [7] readr_1.1.1         tidyr_0.7.2         tibble_1.4.1       
[10] tidyverse_1.2.1     magrittr_1.5        dplyr_0.7.4        
[13] ggplot2_3.0.0      

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-0     repr_0.12.0      reshape2_1.4.3   haven_1.1.0     
 [5] lat

### Specify directories

In [2]:
datadir <- 'downloaded_data' # download data here
if (!dir.exists(datadir)) 
  dir.create(datadir) 

plotsdir <- 'plots' # create plots here
if (!dir.exists(plotsdir)) 
  dir.create(plotsdir) 

## Download data
The cell-gene matrices are availabe on GEO under accession [GSE107947](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE107947).

There are matrices for 10 samples. representing CD45+ and CD45- cells of mice that were mock infected, infected with virus and collected at 48 (2 replicates) or 72 hours, or had a knockout in the key interferon transcription factor IRF7.

Use [GEOquery](https://bioconductor.org/packages/release/bioc/vignettes/GEOquery/inst/doc/GEOquery.html) to download these data and store the names of the downloaded files in a data frame:

In [60]:
# get the GEO experiment
gse <- getGEO("GSE107947", GSEMatrix=FALSE, destdir=datadir)

# download the individual datasets and store downloaded files in data frame
datasets <- lapply(
    gse %>% GSMList %>% names, 
    function (gsm) getGEOSuppFiles(gsm, baseDir=datadir) %>% rownames) %>%
  unlist %>%
  data.frame %>%
  rename_all(function (c) "filename") %>%
  mutate(filename=as.character(filename)) %>% 
  mutate(acc_sample=gsub("_umis.txt.gz", "", basename(filename))) %>%
  separate(acc_sample, c("accession", "sample"), '_', extra="merge")
    
datasets

Using locally cached version of GSE107947 found here:
downloaded_data/GSE107947.soft.gz 
Reading file....
Parsing....
Found 11 entities...
GPL19057 (1 of 12 entities)
GSM2884119 (2 of 12 entities)
GSM2884120 (3 of 12 entities)
GSM2884121 (4 of 12 entities)
GSM2884122 (5 of 12 entities)
GSM2884123 (6 of 12 entities)
GSM2884124 (7 of 12 entities)
GSM2884125 (8 of 12 entities)
GSM2884126 (9 of 12 entities)
GSM2884127 (10 of 12 entities)
GSM2884128 (11 of 12 entities)


filename,accession,sample
downloaded_data/GSM2884119/GSM2884119_PBS_CD45p_treated_umis.txt.gz,GSM2884119,PBS_CD45p_treated
downloaded_data/GSM2884120/GSM2884120_PBS_CD45n_treated_umis.txt.gz,GSM2884120,PBS_CD45n_treated
downloaded_data/GSM2884121/GSM2884121_Flu_treated_CD45p_48h_rep1_umis.txt.gz,GSM2884121,Flu_treated_CD45p_48h_rep1
downloaded_data/GSM2884122/GSM2884122_Flu_treated_CD45n_48h_rep1_umis.txt.gz,GSM2884122,Flu_treated_CD45n_48h_rep1
downloaded_data/GSM2884123/GSM2884123_Flu_treated_CD45p_48h_rep2_umis.txt.gz,GSM2884123,Flu_treated_CD45p_48h_rep2
downloaded_data/GSM2884124/GSM2884124_Flu_treated_CD45n_48h_rep2_umis.txt.gz,GSM2884124,Flu_treated_CD45n_48h_rep2
downloaded_data/GSM2884125/GSM2884125_Flu_treated_CD45p_72h_umis.txt.gz,GSM2884125,Flu_treated_CD45p_72h
downloaded_data/GSM2884126/GSM2884126_Flu_treated_CD45n_72h_umis.txt.gz,GSM2884126,Flu_treated_CD45n_72h
downloaded_data/GSM2884127/GSM2884127_Flu_treated_CD45p_48h_Irf7KO_umis.txt.gz,GSM2884127,Flu_treated_CD45p_48h_Irf7KO
downloaded_data/GSM2884128/GSM2884128_Flu_treated_CD45n_48h_Irf7KO_umis.txt.gz,GSM2884128,Flu_treated_CD45n_48h_Irf7KO


## Read cell-gene matrices
First we read the cell-gene matrices for each sample into an ExpressionSet.

In [119]:
expr_list <- apply(
  datasets,
  1,
  function (dataset) {
    expr_matrix <- read.csv(dataset["filename"], header=TRUE, 
                            row.names=1, sep='\t') %>% 
                   data.matrix
    pData <- expr_matrix %>%
               colnames %>%
               data.frame %>%
               mutate(sample=dataset['sample']) %>%
               remove_rownames %>%
               column_to_rownames(var='.') %>%
               AnnotatedDataFrame
    ExpressionSet(expr_matrix, phenoData=pData)
    }
  )

Then we combine into one merged ExpressionSet:

In [135]:
expr_set <- do.call("combine", expr_list)

In [138]:
expr_set %>% pData %>% group_by(sample) %>% summarize(n()) %>% arrange(sample)

sample,n()
Flu_treated_CD45n_48h_Irf7KO,768
Flu_treated_CD45n_48h_rep1,1152
Flu_treated_CD45n_48h_rep2,1152
Flu_treated_CD45n_72h,384
Flu_treated_CD45p_48h_Irf7KO,768
Flu_treated_CD45p_48h_rep1,1152
Flu_treated_CD45p_48h_rep2,768
Flu_treated_CD45p_72h,768
PBS_CD45n_treated,1152
PBS_CD45p_treated,1152


In [139]:
768 + 1152 * 2 + 384 + 768 + 1152 + 768 * 2

[1] 6912